# Advent of Code 2023

## Day 10 -- Pipe Maze (Part 1)

## Author: Chris Kimber

The instructions for this puzzle can be found at https://adventofcode.com/2023/day/10.

This part is effectively pathfinding, and is something I have no experience with. I looked into the topic a bit during last year's AoC and read about BFS then but I didn't have time to try implementing it then. My solution here uses what is essentially a modified BFS to follow a loop path of "pipes". I used a couple of tutorials on the net for inspiration and did glance at a few solutions on the AoC subreddit, but the solution is my own (and much less elegant than some I saw).

The beginning is just boilerplate to load the data. By splitting each line, the resulting list is effectively a 2D array where list elements are rows and positions within the elements are columns.

In [112]:
with open("input", "r") as f:
    input_grid = f.readlines()

The first dictionary holds the map between the cardinal directions and the change in row-column indices on the array that correspond to moving along the array in that direction.

In [100]:
directions = {"N": (-1, 0), "E": (0, 1), "S": (1, 0), "W": (0, -1)}

In [101]:
pipes = {
    "|": ("N", "S"),
    "-": ("E", "W"),
    "L": ("N", "E"),
    "J": ("N", "W"),
    "7": ("S" ,"W"),
    "F": ("S", "E"),
    ".": (),
    "S": ("N", "E", "W", "S")
}

The starting position in the loop is given by the special character "S"; this saves the start indices. 

In [113]:
for i in range(len(input_grid)):
    for j in range(len(input_grid[i])):
        if input_grid[i][j] == "S":
            start = (i, j)

I chose to break the traversal of the loop up into two steps. The first step is to determine how to leave the start position, which has replaced an ordinary "pipe" in the loop. The shape of the pipe is thus no longer given explicitly, but it can be inferred from the pipes around it. Some solutions I saw dealt with this by manually inspecting the input data and either replacing the corresponding pipe in the input or hardcoding the first move. 

I chose to instead calculate the first move based on the surrounding pipes. As it is a loop, one can move in either direction so the first direction found in the if/else approach is used. The opposite direction returns the same result in the end.

In [123]:
visited_pipes = []

In [124]:
queue = []

In [125]:
i, j = start[0], start[1]
if input_grid[i-1][j] in ["|", "7", "F"]:
    first_direction = "N"
elif input_grid[i+1][j] in ["|", "L", "J"]:
    first_direction = "S"
elif input_grid[i][j+1] in ["-", "L", "F"]:
    first_direction = "E"
else:
    first_direction = "W"
move = directions[first_direction]
new_i, new_j = i + move[0], j + move[1]
visited_pipes.append(start)
queue.append((new_i, new_j))

After determining how to leave the start tile, the rest of the route through the loop is discovered using this BFS-type approach. One can only move two ways at any position, governed by the shape of the pipe. By avoiding already visited positions, this keeps all motion 'forward' through the loop and stops when the loop returns to the start, which is the first element in the list of visited pipes. 

In [127]:
while queue:
    curr = queue.pop(0)
    i, j = curr[0], curr[1]
    curr_pipe = input_grid[i][j]
    for direction in pipes[curr_pipe]:
        move = directions[direction]
        new_i, new_j = i + move[0], j + move[1]
        if (new_i, new_j) in visited_pipes:
            continue
        else:
            visited_pipes.append(curr)
            queue.append((new_i, new_j))
            break

The goal is to find the furthest point in the loop from the start, which will be the halfway point in the loop. If there are an even number of pipes, two different points will be the 'furthest' but if there is an odd, there is a single furthest point. This can be calculated by ceiling division of the length of the list of pipes, here represented as floor division using a negation of the divisor to "invert the floor" to get the ceiling and a negation of the quotient to make the value positive. This is the answer to the problem!

In [128]:
furthest_point = -(len(visited_pipes)//-2)

In [129]:
furthest_point

6828